# Overview
The purpose of this notebook is to implement data preparation and detailed analysis specifically for the chosen modeling task: predict car make from technical attributes (no price or insurance info)

## Modeling Task
The motivating objective for this modeling task is to compare the efficacy of two different approaches for characterising "feature importance" for Random Forest classifiers.  A similar analysis is demonstrated here: https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html 
I will perform a similar set of tasks, but I will do so on the 1985 Auto Imports database.

Upon a cursory reading of the description of the data set (https://www.openml.org/d/9), I noticed (based on intuition) that many of the technical attributes are probably strongly correlated due to constrains of physics, engineering, manufacturing, etc.:

15. engine-type: dohc, dohcv, l, ohc, ohcf, ohcv, rotor.
16. num-of-cylinders: eight, five, four, six, three, twelve, two.
17. engine-size: continuous from 61 to 326.
18. fuel-system: 1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.
19. bore: continuous from 2.54 to 3.94.
20. stroke: continuous from 2.07 to 4.17.
21. compression-ratio: continuous from 7 to 23.
22. horsepower: continuous from 48 to 288.
23. peak-rpm: continuous from 4150 to 6600.

If, in fact, many of these features are correlated, it may have interesting impacts on the experiment.  It may be necessary to take measures to reduce the number of correlated features.

I'm specifically intersted in doing a demonstration of a multi-class classifier.  Therefore, I will use the "make" attribute (attribute 3) as the response variable.

Fore model features, initially, I will consider using all features other than the monetary-related features (1. symboling; 2. normalized-losses; 26. price)

## TODO
- Consider custom feature encoding
- Transform data into supervised learning structure: reponse vector + design matrix
- Make partitions for Train, Test, Validation (note, there are only 205 rows in whole data set)